# Notebook 03: Diagnostic Matching

## Cell 1 — Purpose + scope (diagnostics only)

In [7]:
# Purpose:
# - Isolate diagnostic checks that support limitations and data-linkage validity.
# - Quantify unmatched sales after deterministic Sales → Lead matching.
# - Keep this notebook separate from modelling and descriptive profiling to avoid duplication.

# Define the diagnostic scope for traceability when the notebook is executed
analysis_scope = "diagnostics_matching_only"  # Signals that this notebook focuses only on matching diagnostics

# Print the scope so the notebook output is self-explanatory when reviewed
print("Notebook 03 scope:", analysis_scope)   # Confirms this notebook is not used for modelling or EDA

Notebook 03 scope: diagnostics_matching_only


## Cell 2 — Imports + output directory

In [10]:
# Purpose:
# - Import only the libraries required for diagnostics and output saving.
# - Create a dedicated outputs folder for Notebook 03 to keep artefacts organised for GitHub.

import os                 # Enables folder creation and portable file path handling
import pandas as pd        # Provides DataFrame functionality for reading and summarising diagnostic tables

# Define a dedicated folder for Notebook 03 outputs
output_dir_03 = "outputs_03_diagnostics"      # Keeps diagnostic artefacts separate from modelling and EDA outputs

# Create the output folder if it does not already exist
os.makedirs(output_dir_03, exist_ok=True)     # Ensures saving files does not fail due to missing directories

# Confirm where diagnostic outputs will be written
print("Notebook 03 outputs will be saved to:", output_dir_03)  # Traceability for downstream saved tables


Notebook 03 outputs will be saved to: outputs_03_diagnostics


## Cell 3 — Quantify matched vs unmatched sales

In [18]:
# Cell 3 — Load client-supplied unmatched sales diagnostic extract
# Purpose:
# - Load the client-provided dataset of sales records that could not be linked to a lead in the Leads tab.
# - Quantify the scale of unmatched sales as a data-linkage limitation.
# - Preserve the client-provided 'reason' field for transparent diagnostic reporting.

# Define the expected input file name for the client-supplied unmatched sales extract
UNMATCHED_SALES_FILE = "sales_unmatched_after_join.csv"  # File contains unmatched sales and a client-provided reason label

# Load the unmatched sales diagnostic dataset
sales_unmatched = pd.read_csv(
    UNMATCHED_SALES_FILE
)  # Reads the client extract into a DataFrame for summarisation and reporting

# Count the number of unmatched sales records
n_unmatched_sales = len(sales_unmatched)  # Each row represents one sale record that was not linked to a lead

# Display the number of unmatched sales for traceability
print("Unmatched sales records (client extract):", n_unmatched_sales)  # Establishes the scale of unmatched sales


Unmatched sales records (client extract): 262


## Cell 4 — Breakdown of unmatched sales by reason

In [21]:
# Purpose:
# - Summarise why sales could not be linked to leads, using the client-provided reason labels.
# - Quantify the relative contribution of each failure mode.
# - Provide a clear diagnostic table that can be referenced as a limitation.

# Count unmatched sales by client-provided reason
unmatched_reason_summary = (
    sales_unmatched
    .groupby("reason", dropna=False)          # Group unmatched sales by stated reason
    .size()                                   # Count number of sales per reason
    .reset_index(name="count")                # Convert counts to a tidy table
    .sort_values("count", ascending=False)    # Rank reasons by frequency
)

# Calculate percentage share of each reason
total_unmatched = unmatched_reason_summary["count"].sum()   # Total unmatched sales for denominator
unmatched_reason_summary["percentage"] = (
    unmatched_reason_summary["count"] / total_unmatched * 100
)  # Express each reason as a percentage of unmatched sales

# Display diagnostic summary
print("Unmatched sales by reason:")
print(unmatched_reason_summary)


Unmatched sales by reason:
               reason  count  percentage
0  phone not in leads    262       100.0


## Cell 5 — Save unmatched sales diagnostic summary

In [24]:
# Purpose:
# - Persist the diagnostic breakdown of unmatched sales for reference and reporting.
# - Ensure the limitation is backed by a reproducible table.

# Save unmatched reason summary to disk
output_path = os.path.join(
    output_dir_03,
    "unmatched_sales_reason_summary.csv"
)

unmatched_reason_summary.to_csv(
    output_path,
    index=False
)

print("Unmatched sales diagnostic summary saved to:", output_path)


Unmatched sales diagnostic summary saved to: outputs_03_diagnostics/unmatched_sales_reason_summary.csv


## Cell 6 — Verify whether unmatched sales identifiers exist in the leads universe

In [27]:
# Purpose:
# - Independently validate whether unmatched sales could be linked via email or phone identifiers.
# - Confirm whether the client-provided reason ("phone not in leads") is consistent with the leads universe.
# - Avoid re-running matching logic by checking key presence only.

# Load the prepared lead-level dataset created in Notebook 01
lead_model = pd.read_csv(
    "lead_model_final.csv"
)  # Provides the leads universe used for modelling, including standardised keys (if saved)

# --- Email presence check ---
# Standardise sales emails to a comparable key format
sales_unmatched["email_key_sales_check"] = (
    sales_unmatched["email_id"].astype(str).str.strip().str.lower()
)  # Creates a comparable email key from unmatched sales

# Ensure lead email keys exist; if not present, derive from raw email column
if "email_key" in lead_model.columns:
    lead_model["email_key_check"] = lead_model["email_key"].astype(str).str.strip().str.lower()
else:
    lead_model["email_key_check"] = lead_model["email"].astype(str).str.strip().str.lower()  # Fallback if email_key was not saved

# Build sets for fast membership checking
lead_email_set = set(lead_model["email_key_check"].replace({"nan": None}).dropna().unique())  # Unique lead emails
unmatched_email_set = set(sales_unmatched["email_key_sales_check"].replace({"nan": None}).dropna().unique())  # Unique unmatched sale emails

# Compute overlap
email_overlap = unmatched_email_set & lead_email_set  # Emails present in both unmatched sales and leads

# Report email overlap
print("Email linkage feasibility check:")
print("Unique unmatched sale emails:", len(unmatched_email_set))
print("Unmatched sale emails present in leads:", len(email_overlap),
      f"({len(email_overlap)/max(1, len(unmatched_email_set)):.2%})")

# --- Phone presence check (optional but consistent with the 'reason' field) ---
# Only run if the necessary key columns exist in both datasets
if "phone_key" in sales_unmatched.columns and "phone_key" in lead_model.columns:
    lead_phone_set = set(lead_model["phone_key"].astype(str).replace({"nan": None}).dropna().unique())  # Unique lead phone keys
    unmatched_phone_set = set(sales_unmatched["phone_key"].astype(str).replace({"nan": None}).dropna().unique())  # Unique unmatched phone keys
    phone_overlap = unmatched_phone_set & lead_phone_set  # Phones present in both unmatched sales and leads

    print("\nPhone linkage feasibility check:")
    print("Unique unmatched sale phone keys:", len(unmatched_phone_set))
    print("Unmatched sale phone keys present in leads:", len(phone_overlap),
          f"({len(phone_overlap)/max(1, len(unmatched_phone_set)):.2%})")
else:
    print("\nPhone linkage feasibility check skipped (phone_key column not available in both datasets).")


Email linkage feasibility check:
Unique unmatched sale emails: 247
Unmatched sale emails present in leads: 0 (0.00%)

Phone linkage feasibility check:
Unique unmatched sale phone keys: 247
Unmatched sale phone keys present in leads: 0 (0.00%)


## Cell 7 — Save linkage feasibility check summary

In [32]:
# Purpose:
# - Persist verification that unmatched sales cannot be linked to leads via phone or email.
# - Provide a reproducible audit trail supporting exclusion of these records.

linkage_check_summary = pd.DataFrame([{
    "unmatched_sales": len(unmatched_email_set),
    "email_overlap_count": len(email_overlap),
    "email_overlap_pct": len(email_overlap) / max(1, len(unmatched_email_set)) * 100,
    "phone_overlap_count": len(phone_overlap),
    "phone_overlap_pct": len(phone_overlap) / max(1, len(unmatched_phone_set)) * 100
}])

output_path = os.path.join(
    output_dir_03,
    "unmatched_sales_linkage_feasibility_check.csv"
)

linkage_check_summary.to_csv(output_path, index=False)
print("Linkage feasibility check saved to:", output_path)


Linkage feasibility check saved to: outputs_03_diagnostics/unmatched_sales_linkage_feasibility_check.csv
